<a href="https://colab.research.google.com/github/Madhu-712/Chip_designer_copilot_Agent/blob/main/Autogen/MultiAgent_Autogen_chipdesign_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Autogen Multiagent Chipdesign Agent
###AutoGen allows the orchestration of multiple LLMs, enabling specialized agents to collaborate on solving complex tasks. In this post, we’ll develop a use case focusing on optimizing an chipdesign architecture which helps semiconductor industry to improvise on the performance of chip architecture and get real time designing insights .

In [ ]:
!pip install autogen

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.9 MB/s eta 0:00:00


In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
subprocess.Popen(["ollama", "serve"])

<Popen: returncode: None args: ['ollama', 'serve']>

In [ ]:
!ollama pull llama3.2:1b

pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠇ pulling manifest ⠇ pulling manifest 
pulling 74701a8c35f6:   1% ▕▏  11 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   3% ▕▏  38 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   6% ▕▏  85 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:   9% ▕▏ 124 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  11% ▕▏ 145 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  13% ▕▏ 176 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  15% ▕▏ 204 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  17% ▕▏ 229 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  21% ▕▏ 272 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:  25% ▕▏ 329 MB/1.3 GB                  pulling manifest 
pulling 74701a8c35f6:

In [ ]:
import os

os.environ['OAI_CONFIG_LIST'] ='[{"model": "llama3.2:1b","api_key": "EMPTY", "max_tokens":1000}]'

In [ ]:
import autogen

llm_config={
    "timeout": 600,
    "cache_seed": 68,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["llama3.2:1b"]},
    ),
    "temperature": 0.5,
}

llm_config['config_list'][0]["base_url"] = f"http://localhost:11434/v1"

In [ ]:
!pip install chromadb
import os
from typing import Any, Callable, Dict, List, Optional, Tuple, Type, Union

import chromadb
from PIL import Image
from termcolor import colored

import autogen
from autogen import Agent, AssistantAgent, ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.img_utils import _to_pil, get_image_data
from autogen.agentchat.contrib.multimodal_conversable_agent import MultimodalConversableAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.code_utils import DEFAULT_MODEL, UNKNOWN, content_str, execute_code, extract_code, infer_lang

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import autogen
from autogen import UserProxyAgent, register_function, AssistantAgent

#Termination Policy

In [ ]:
def termination_message(msg: str) -> bool:
    content = msg.get("content", "").lower()
    return "terminate" in content

#Agents

In [ ]:
# The user proxy agent that executes tool calls.
user_proxy = UserProxyAgent(
    name="User",
    llm_config=False,
    is_termination_msg=termination_message,
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
)

In [ ]:
Orchestrator =AssistantAgent(
    name="OrchestratorAgent",
    description="Orchestrates the interactions between AI model Agent and Design Agent.The Orchestrator agent maintains the overall quality and consistency of Final design",
    system_message="""Manage the entire AI-assisted chip design workflow for an IT startup ensuring seamless integration of AI suggestions and maintaining design integrity.""",
    llm_config=llm_config,
)

In [ ]:

AI_model =AssistantAgent(
    name="AI_modelAgent",
    description="This Agent analyze the provided chip design data using pre-trained AI models to identify potential bugs, suggest optimizations ,Power consumption, area, performance,reliability and cost. Provide predictions.",
    system_message="""Analyze chip design data using pre-trained AI models and provide actionable suggestions for bug detection,optimization, and other improvements.A structured report including flagged potential bugs, specific optimization suggestions (with justifications), and predictions """,

    llm_config=llm_config,
)

In [ ]:
Design=AssistantAgent(
    name="Design_assistant_agent",
    description ="This agent possesses deep expertise in EDA (Electronic design automation)tools and the intricacies of chip design,chip architecture.It acts as a gatekeeper, carefully reviewing AI suggestions before they are integrated.The Design Agent prioritizes design quality and reliability, preventing the introduction of errors or undesirable trade-offs.",
    system_message="""Act like a Design Assistant or coworker. Evaluate AI-generated suggestions for feasibility, correctness and compliance with design constraint ensuring the design's integrity and functionality.""",
    llm_config=llm_config,
)

In [ ]:
groupchat = autogen.GroupChat(agents=[user_proxy, Orchestrator, AI_model,Design], messages=[], max_round=6)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [ ]:
user_proxy.initiate_chat(manager, message="""
Generate a report on chip designing architecture detailing power,manufacturing complexity, pin count optimization techniques""")

User (to chat_manager):


Generate a report on chip designing architecture detailing power,manufacturing complexity, pin count optimization techniques

--------------------------------------------------------------------------------
[autogen.oai.client: 05-01 16:17:04] {696} WARNING - Model llama3.2:1b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


[autogen.oai.client: 05-01 16:17:05] {696} WARNING - Model llama3.2:1b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


[autogen.oai.client: 05-01 16:17:06] {696} WARNING - Model llama3.2:1b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.



Next speaker: User

User (to chat_manager):



--------------------------------------------------------------------------------
[autogen.oai.client: 05-01 16:17:11] {696} WARNING - Model llama3.2:1b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


[autogen.oai.client: 05-01 16:17:12] {696} WARNING - Model llama3.2:1b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


...


Next speaker: User

User (to chat_manager):



--------------------------------------------------------------------------------
[autogen.oai.client: 05-01 16:17:22] {696} WARNING - Model llama3.2:1b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.



Next speaker: User

User (to chat_manager):



--------------------------------------------------------------------------------
[autogen.oai.client: 05-01 16:17:24] {696} WARNING - Model llama3.2:1b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.


[autogen.oai.client: 05-01 16:17:24] {696} WARNING - Model llama3.2:1b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.



Next speaker: User

User (to chat_manager):



--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (388c9c20-70ff-4eae-9aaf-287d5999e74c): Maximum rounds (6) reached


ChatResult(chat_id=None, chat_history=[{'content': '\nGenerate a report on chip designing architecture detailing power,manufacturing complexity, pin count optimization techniques', 'role': 'assistant', 'name': 'User'}, {'content': '', 'role': 'assistant', 'name': 'User'}, {'content': '**Chip Design Architecture Report: Power, Manufacturing Complexity, and Pin Count Optimization Techniques**\n\n**Executive Summary**\n\nThe goal of this report is to evaluate the feasibility, correctness, and compliance with design constraints of various chip design architectures. We will assess the power consumption, manufacturing complexity, and pin count optimization techniques used in different architectures.\n\n**Introduction**\n\nDesigning microprocessors and other integrated circuits (ICs) requires balancing competing factors such as performance, power consumption, manufacturing complexity, and pin count. This report provides an overview of several popular chip design architectures, including ARMv7

#streamlit code

In [ ]:

import streamlit as st
import autogen
import os
import subprocess
import warnings

# Install necessary packages if not already installed
try:
    import autogen
except ImportError:
    st.write("Installing autogen...")
    subprocess.run(["pip", "install", "autogen"])
    import autogen

# Ollama installation and setup
try:
    import ollama
except ImportError:
    st.write("Installing and setting up Ollama...")
    subprocess.run(["curl", "-fsSL", "https://ollama.com/install.sh", "|", "sh"])
    import ollama

subprocess.Popen(["ollama", "serve"])
subprocess.run(["ollama", "pull", "llama3.2:1b"])

os.environ['OAI_CONFIG_LIST'] ='[{"model": "llama3.2:1b","api_key": "EMPTY", "max_tokens":1000}]'

# AutoGen configuration
llm_config={
    "timeout": 600,
    "cache_seed": 68,
    "config_list": autogen.config_list_from_json(
        "OAI_CONFIG_LIST",
        filter_dict={"model": ["llama3.2:1b"]},
    ),
    "temperature": 0.5,
}

llm_config['config_list'][0]["base_url"] = f"http://localhost:11434/v1"

# Suppress warnings
warnings.filterwarnings("ignore")

# Agent definitions
from autogen import UserProxyAgent, AssistantAgent

def termination_message(msg: str) -> bool:
    content = msg.get("content", "").lower()
    return "terminate" in content

user_proxy = UserProxyAgent(
    name="User",
    llm_config=False,
    is_termination_msg=termination_message,
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
)

Orchestrator = AssistantAgent(
    name="OrchestratorAgent",
    description="Orchestrates the interactions between AI model Agent and Design Agent.The Orchestrator agent maintains the overall quality and consistency of Final design",
    system_message="""Manage the entire AI-assisted chip design workflow for an IT startup ensuring seamless integration of AI suggestions and maintaining design integrity.""",
    llm_config=llm_config,
)

AI_model = AssistantAgent(
    name="AI_modelAgent",
    description="This Agent analyze the provided chip design data using pre-trained AI models to identify potential bugs, suggest optimizations ,Power consumption, area, performance,reliability and cost. Provide predictions.",
    system_message="""Analyze chip design data using pre-trained AI models and provide actionable suggestions for bug detection,optimization, and other improvements.A structured report including flagged potential bugs, specific optimization suggestions (with justifications), and predictions """,
    llm_config=llm_config,
)

Design = AssistantAgent(
    name="Design_assistant_agent",
    description ="This agent possesses deep expertise in EDA (Electronic design automation)tools and the intricacies of chip design,chip architecture.It acts as a gatekeeper, carefully reviewing AI suggestions before they are integrated.The Design Agent prioritizes design quality and reliability, preventing the introduction of errors or undesirable trade-offs.",
    system_message="""Act like a Design Assistant or coworker. Evaluate AI-generated suggestions for feasibility, correctness and compliance with design constraint ensuring the design's integrity and functionality.""",
    llm_config=llm_config,
)

groupchat = autogen.GroupChat(agents=[user_proxy, Orchestrator, AI_model, Design], messages=[], max_round=6)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

# Streamlit UI
st.title("Chip Design Assistant")

user_input = st.text_area("Enter your chip design query:")

if st.button("Submit"):
    with st.spinner("Generating report..."):
        response = user_proxy.initiate_chat(
            manager, message=user_input
        )
        st.write(response)

In [ ]:
#requirements.txt
autogen
streamlit
chromadb
Pillow # PIL (Pillow) is required by autogen
termcolor
ollama